Analyzing time for each step in tracking.process_im function.

Conclusion: Most time-intensive step was filtering blobs, which varied significantly depending on how many blobs the segmentation function returned.

In [52]:
import time
import numpy as np
import pandas as pd
import os

import skimage.io
import skimage.morphology
import scipy.spatial.distance
import tracking

import bootcamp_utils
from behavioral_analysis import segmentation as seg
import skimage.morphology

In [2]:
_kw_dict = {'c': 'centroid',
            'e': 'eccentricity',
            'j': 'major_axis_length',
            'n': 'minor_axis_length',
            'o': 'orientation',
            'p': 'perimeter',
            's': 'area'}

In [75]:
fdir = '//scepto.caltech.edu/Parker_lab/Christina_Julian/8_10_2018/'

allfiles = os.listdir(fdir)
im_names = [f for f in allfiles if '.tiff' in f]
im_names = im_names[::600]
ims = [skimage.io.imread(fdir + f) for f in im_names]

KeyboardInterrupt: 

In [109]:
im_bg = skimage.io.imread('C:/Users/1004p/Desktop/Caltech Amgen 2018/Behavioral Analysis/sceptovideo/im_bg.tiff', as_gray = True)
im = ims[1]

In [110]:
s1 = time.perf_counter()
s2 = time.process_time()

im = seg.bg_subtract(im, im_bg)

f1 = time.perf_counter()
f2 = time.process_time()

print(f1-s1)
print(f2-s2)

0.5502456291451381
0.8125


In [111]:
s1 = time.perf_counter()
s2 = time.process_time()

im_bw, im_labeled, n_labels = seg.segment(im)
print(n_labels)

im = skimage.morphology.remove_small_objects(im_labeled, min_size = 200, connectivity = 2) + 0
im_labeled, n_labels = skimage.measure.label(im, return_num=True)
print(n_labels)

f1 = time.perf_counter()
f2 = time.process_time()
print(f1-s1)
print(f2-s2)

70195
13
0.5123644200211857
0.5


In [112]:
s1 = time.perf_counter()
s2 = time.process_time()

#specs is a list of all specimen [label, size, coordinates]
specs = []
properties = skimage.measure.regionprops(im_labeled)

f1 = time.perf_counter()
f2 = time.process_time()
print(f1-s1)
print(f2-s2)

0.04626756292873324
0.046875


In [113]:
include_frame = True
frame_id = 0
props = 'scjn'

s1 = time.perf_counter()
s2 = time.process_time()

for i in range(n_labels):
    if True:
        blob = (im_labeled == (i+1)) + 0
        if tracking.thresholding(blob, thresh_list=[(200, 9000), (0.88, 1)]):
            spec_data = [properties[i][_kw_dict[x]] for x in props]
            if include_frame:
                spec_data.insert(0, frame_id)
        else:
            continue
    else:
        spec_data = [properties[i][_kw_dict[x]] for x in props]
        if include_frame:
            spec_data.insert(0, frame_id)
    specs.append(spec_data)

f1 = time.perf_counter()
f2 = time.process_time()
print(f1-s1)
print(f2-s2)

C:\ProgramData\Anaconda3\lib\site-packages\skimage\measure\_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
C:\ProgramData\Anaconda3\lib\site-packages\skimage\measure\_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)


0.6986277557971334
1.0625


In [118]:
print(spec_data)
df = pd.DataFrame(data=specs, columns=col_names)
df1 = df.reindex(columns = col_names + ['x', 'y'])
df1[['x', 'y']] = df['centroid'].apply(pd.Series)
print(df1.head())

[0, 255, (1992.4901960784314, 2.4784313725490197), 47.700264345491156, 7.508512568644098]
   frame  area                                  centroid  major_axis_length  \
0      0  1605   (1.5246105919003115, 570.9520249221184)         473.911629   
1      0  2231  (1.6024204392649037, 1421.7669206633796)         743.662347   
2      0   253   (44.72727272727273, 3.6284584980237153)          35.012871   
3      0   255  (1992.4901960784314, 2.4784313725490197)          47.700264   

   minor_axis_length            x            y  
0           4.847631     1.524611   570.952025  
1           4.903344     1.602420  1421.766921  
2          10.155144    44.727273     3.628458  
3           7.508513  1992.490196     2.478431  


In [114]:
s1 = time.perf_counter()
s2 = time.process_time()

col_names = [_kw_dict[x] for x in props]
if include_frame:
    col_names.insert(0, 'frame')

df = pd.DataFrame(data=specs, columns=col_names)
df[['x', 'y']] = df['centroid'].apply(pd.Series)
df.drop('centroid', axis = 1, inplace = True)

f1 = time.perf_counter()
f2 = time.process_time()
print(f1-s1)
print(f2-s2)

0.008488279529956344
0.0


In [33]:
sep_centroid = True

s1 = time.perf_counter()
s2 = time.process_time()

if type(im) != np.ndarray:
    raise Exception('im must be a 2d numpy ndarray.')
if len(props) != len(set(props)):
    print('WARNING: Some property keywords are repeated. Redundancies were removed.')
    props = list(sorted(set(props), key=props.index))
if type(include_frame) != bool:
    raise Exception('include_frame must be a boolean.')
if type(im_bg) != np.ndarray:
    if im_bg == None:
        pass
    else:
        raise Exception('im_bg must be a 2d numpy ndarray.')
if sep_centroid and 'c' not in props:
    raise Exception('There is no centroid data to separate.')

f1 = time.perf_counter()
f2 = time.process_time()
print(f1-s1)
print(f2-s2)

0.0001900243159980164
0.0


In [115]:
df

,frame,area,major_axis_length,minor_axis_length,x,y
0,0,1605,473.911629,4.847631,1.524611,570.952025
1,0,2231,743.662347,4.903344,1.602420,1421.766921
2,0,253,35.012871,10.155144,44.727273,3.628458
3,0,255,47.700264,7.508513,1992.490196,2.478431
